# Climate Sensitivity atmospheric plots of the HIGH and LOW CESM runs

## Baatsen _et al._ (2018) Fig. 14

a. climate response map $[^\circ C / W m^{-2}]$ (winter season only)

b. climate response zonal mean $[^\circ C / W m^{-2}]$ (annual, DJF, JJA)

c. map of change in net LW flux $[W m^{-2}]$ + SW fluxes

d. zonal mean TOM fluxes $[W m^{-2}]$ 

1. SW total     `FSNT`    = "Net solar flux at top of model" or 
                `FSNTOA`  = "Net solar flux at top of atmosphere"
2. SW clearsky  `FSNTC`   = "Clearsky net solar flux at top of model" or 
                `FSNTOAC` = "Clearsky net solar flux at top of atmosphere"
3. LW total     `FLNT`    = "Net longwave flux at top of model"
4. LW clearsky  `FLNTC` = "Clearsky net longwave flux at top of model"

e. (occurance deep convection $[\%]$ `FREQZM` = "Fractional occurance of ZM convection")

f. zonal mean cloud cover $[\%]$

1. low `CLDLOW`
2. medium `CLDMED`
3. high `CLDHGH`
4. total `CLDTOT` = "Vertically-integrated total cloud"

## other quantities
- heat transport:  total = atm. + ocean
- `PRECT` = "Total (convective and large-scale) precipitation rate (liq + ice)"

## comparisons: HIGH vs. LOW
1. (2090-2100 RCP) - (200-230 CTRL)
2. (2040-2050 4xCO2) - (200-230 CTRL)

In [ ]:
import os
import sys
import numpy as np
import string
import xarray as xr
import cmocean
import cartopy
import scipy.stats as stats
import cartopy.crs as ccrs
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
mpl.rc_file('../rc_file')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append("..")
from tqdm.notebook import tqdm
from paths import path_results, path_prace, CESM_filename, file_ex_atm_lpd
from filters import lowpass
from constants import spy
from xr_DataArrays import xr_AREA
from xr_regression import xr_linear_trend, xr_lintrend

created yearly files in timeseries.ipynb with relevant 
```
fieldlist = [['CLDLOW','CLDMED','CLDHGH','CLDTOT'],['FSNT','FSNTC','FLNT','FLNTC'],['FREQZM','PRECT']]
for fields in tqdm(fieldlist):
    for i, run in tqdm(enumerate(['ctrl','lpd','hq','lq','rcp','lr1'])):
        ys = [200,500,2041,2041,2091,2091][i]
        ye = [230,530,2051,2051,2101,2101][i]
        DeriveField(run).yrly_avg_nc(domain='atm', fields=fields, years=np.arange(ys,ye))
```
for lpd it created all files form 154 to 501, so we use np.arange(470,500) as avearging period and delete the other files

In [ ]:
# timeseries created files for all years, deleted unnecessary files
# for q in ['CLDLOW_CLDMED_CLDHGH_CLDTOT', 'FSNT_FSNTC_FLNT_FLNTC', 'FREQZM_PRECT']:
#     for y in tqdm(np.arange(154, 502)):
#         if y not in np.arange(470,500):
#             fn_in = f'{path_prace}/lpd/atm_yrly_{q}_{y:04d}.nc'
#             if os.path.exists(fn_in):
#                 os.remove(fn_in)

In [ ]:
for q in ['CLDLOW_CLDMED_CLDHGH_CLDTOT', 'FSNT_FSNTC_FLNT_FLNTC', 'FREQZM_PRECT']:
    for i, run in tqdm(enumerate(['ctrl','lpd','hq','lq','rcp','lr1'])):
        # if i==1:  continue
        ys = [200,470,2041,2041,2091,2091][i]
        ye = [230,500,2051,2051,2101,2101][i]
        fn_in = f'{path_prace}/{run}/atm_yrly_{q}_*.nc'
        fn_out = f'{path_prace}/ECS/atm_{run}_{q}_avg_{ys}_{ye}.nc'
        if os.path.exists(fn_out):  continue
        xr.open_mfdataset(fn_in, concat_dim='time').mean('time').to_netcdf(fn_out)

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(6.4,3), sharey=True)
ax[0].set_ylabel('latitude  [$^\circ$N]')
ax[1].axvline(0, c='k', lw=.5)
for i in range(3):
    ax[i].axhline(0, c='k',lw=.5)
    ax[i].text(.01,.94,f'({string.ascii_lowercase[i]})', transform=ax[i].transAxes)
    # ax[i].set_title(['clouds','radiation'][i])
    # ax[i].axvline(0, lw=.5, c='k')
for i, sim in enumerate(['HIGH', 'LOW']):
    ls = ['-','--'][i]
    area = xr_AREA(['atm', 'atm_f09'][i])
    (ctrl, rcp, quad) = [('ctrl', 'rcp', 'hq'), ('lpd', 'lr1', 'lq')][i]
    ysc = [200,470][i]
    for q, Q in enumerate(['CLDLOW_CLDMED_CLDHGH_CLDTOT','FSNT_FSNTC_FLNT_FLNTC']):
        dsc = xr.open_dataset(f'{path_prace}/ECS/atm_{ctrl}_{Q}_avg_{ysc}_{ysc+30}.nc')
        dsr = xr.open_dataset(f'{path_prace}/ECS/atm_{rcp}_{Q}_avg_2091_2101.nc')
        dsq = xr.open_dataset(f'{path_prace}/ECS/atm_{quad}_{Q}_avg_2041_2051.nc')
        if q==0:  # clouds
            for t, T in enumerate(['CLDLOW','CLDMED','CLDHGH','CLDTOT']):
                label = [['low', 'medium', 'high', 'total'][t], None][i]
                c = f'C{t}'
                ax[2].plot(dsc[T].mean('lon')*100, dsc.lat, c=c, ls=ls, label=label)
        if q==1:  # radiation
            for t, T in enumerate(['FSNT','FSNTC','FLNT','FLNTC']):
                label = [['SW net', 'SW clearsky', 'LW net', 'LW clearsky'][t], None][i]
                c = f'C{t}'
                ax[0].plot(dsc[T].mean('lon'), dsc.lat, c=c, ls=ls, label=label)
            ax[1].plot(((dsc['FSNT']-dsc['FLNT'])*area).sum('lon').cumsum('lat')/1e15, dsc.lat, c='k', ls=ls, label='total heat flux')
for i in range(3):
    ax[i].legend(fontsize=7)
ax[2].set_xlim((0,100))
ax[0].set_xlabel(r'radiation  [W/m$^2$]')
ax[1].set_xlabel('heat transport  [PW]')
ax[2].set_xlabel('cloud fraction  [%]')
# ax[0].set_xlim((0,89))
ax[0].set_ylim((-90,90))
ax[0].set_yticks(np.arange(-90,91,30))
plt.savefig(f'{path_results}/ECS/cloud_radiation_mean')

In [ ]:
f, ax = plt.subplots(1,4, figsize=(6.4,3), sharey=True)
# for i in range(3):
#     ax[0,i].set_title(['CTRL', 'RCP 2091-2100 - CTRL', '4xCO2 2041-2050 - CTRL'][i])
#     ax[i,0].set_ylabel(['clouds','radiation'][i]+'\nlatitude')
#     for j in range(2):  ax[i,j+1].axvline(0, lw=.5, c='k')
for i in range(4):
    ax[i].axhline(0, c='k',lw=.5)
    ax[i].axvline(0, c='k',lw=.5)
    ax[i].text(.01,.94,f'({string.ascii_lowercase[i]})', transform=ax[i].transAxes)

for i, sim in enumerate(['HIGH', 'LOW']):
    ls = ['-','--'][i]
    (ctrl, rcp, quad) = [('ctrl', 'rcp', 'hq'), ('lpd', 'lr1', 'lq')][i]
    ysc = [200,470][i]
    Q = 'CLDLOW_CLDMED_CLDHGH_CLDTOT'
    dsc = xr.open_dataset(f'{path_prace}/ECS/atm_{ctrl}_{Q}_avg_{ysc}_{ysc+30}.nc')
    dsr = xr.open_dataset(f'{path_prace}/ECS/atm_{rcp}_{Q}_avg_2091_2101.nc')
    dsq = xr.open_dataset(f'{path_prace}/ECS/atm_{quad}_{Q}_avg_2041_2051.nc')
    for t, T in enumerate(['CLDLOW','CLDMED','CLDHGH','CLDTOT']):
        if i==0:  ax[t].set_title(['low clouds', 'medium clouds', 'high clouds', 'total clouds'][t])
        c = f'C{t}'
        control = dsc[T].mean('lon')
        ax[t].plot((dsr[T].mean('lon')-control)*100, dsc.lat, c=c, ls=ls, lw=0.7)
        ax[t].plot((dsq[T].mean('lon')-control)*100, dsc.lat, c=c, ls=ls, lw=1.5)
        if i==0:  ax[t].set_xlabel('change  [%]')
ax[0].set_ylim((-90,90))
ax[0].set_yticks(np.arange(-90,91,30))
ax[0].set_ylabel(r'latitude  [$^\circ$N]')
plt.savefig(f'{path_results}/ECS/ECS_cloud_change')

In [ ]:
f, ax = plt.subplots(3,3, figsize=(6.4,6.4), sharey=True)
for i in range(3):
    ax[i,0].set_ylabel(['SW radiation in', 'LW radiation out', 'total radiation in'][i]+'\nlatitude  [$^\circ$N]')
    ax[0,i].set_title(['clearsky', 'with clouds', 'cloud effect'][i])
    ax[2,i].set_xlabel(r'radiation change [W/m$^2$]')
    for j in range(3):
        ax[i,j].text(.01,.92,f'({string.ascii_lowercase[3*i+j]})', transform=ax[i,j].transAxes)
        ax[i,j].axvline(0, c='k',lw=.5)
        ax[i,j].axhline(0, c='k',lw=.5)

for i, sim in enumerate(['HIGH', 'LOW']):
    ls = ['-','--'][i]
    (ctrl, rcp, quad) = [('ctrl', 'rcp', 'hq'), ('lpd', 'lr1', 'lq')][i]
    ysc = [200,470][i]
    Q = 'FSNT_FSNTC_FLNT_FLNTC'
    dsc = xr.open_dataset(f'{path_prace}/ECS/atm_{ctrl}_{Q}_avg_{ysc}_{ysc+30}.nc')
    dsr = xr.open_dataset(f'{path_prace}/ECS/atm_{rcp}_{Q}_avg_2091_2101.nc')
    dsq = xr.open_dataset(f'{path_prace}/ECS/atm_{quad}_{Q}_avg_2041_2051.nc')
    for ds in [dsc, dsr, dsq]:
        ds['TOM']  = ds['FSNT']  - ds['FLNT']   # top of model radiative imbalance
        ds['TOMC'] = ds['FSNTC'] - ds['FLNTC']  # top of model clearsky radiative imbalance
    for k, ds_ in enumerate([dsr, dsq]):
        kwargs = {'c':['C1','C3'][k], 'ls':['-','--'][i], 'lw':.8, 'label':['HIGH ','LOW '][i]+['RCP','4xCO2'][k]}
        for t, T in enumerate(['FSNTC','FSNT','FLNTC','FLNT']):
            ax[int(t/2),t%2].plot(ds_[T].mean('lon')-dsc[T].mean('lon'), dsc.lat, **kwargs)
        for j in range(2):
            # cloud effects
            (nt, ntc) = [('FSNT', 'FSNTC'),('FLNT', 'FLNTC')][j]
            ax[j,2].plot((ds_[nt]-dsc[nt]-(ds_[ntc]-dsc[ntc])).mean('lon'), dsc.lat, **kwargs)
            # total radiation
            T = ['TOMC','TOM'][j]
            ax[2,j].plot(ds_[T].mean('lon')-dsc[T].mean('lon'), dsc.lat, **kwargs)
        ax[2,2].plot((ds_['TOM']-dsc['TOM']-(ds_['TOMC']-dsc['TOMC'])).mean('lon'), dsc.lat, **kwargs)
for i in range(3):
    ax[i,0].set_ylim((-90,90))
    ax[i,0].set_yticks(np.arange(-90,91,30))
ax[1,0].legend(fontsize=7)
plt.savefig(f'{path_results}/ECS/ECS_clouds_clearsky_radiation_lat')

- net LW clearsky: Planck response, polar amplification due to enhanced warming, tropical tropopause response and stratopsheric cooling result in less outgoing radiation
- net LW with cloud effects
- SW clearsky: albedo changes on surface
### common features:
- RCP response less strong than 4xCO2
- total cloud cover reduces in low- and mid-latitudes
- low clouds decrease everywhere, except Arctic

In [ ]:
f, ax = plt.subplots(3,3, figsize=(10,10), sharey=True)
for i in range(3):
    ax[0,i].set_title(['CTRL', 'RCP 2091-2100 - CTRL', '4xCO2 2041-2050 - CTRL'][i])
    ax[i,0].set_ylabel(['clouds','radiation', 'precip'][i]+'\nlatitude')
    for j in range(2):  ax[i,j+1].axvline(0, lw=.5, c='k')
for i, sim in enumerate(['HIGH', 'LOW']):
    ls = ['-','--'][i]
    (ctrl, rcp, quad) = [('ctrl', 'rcp', 'hq'), ('lpd', 'lr1', 'lq')][i]
    ysc = [200,470][i]
    for q, Q in enumerate(['CLDLOW_CLDMED_CLDHGH_CLDTOT',
                           'FSNT_FSNTC_FLNT_FLNTC', 'FREQZM_PRECT']):
        dsc = xr.open_dataset(f'{path_prace}/ECS/atm_{ctrl}_{Q}_avg_{ysc}_{ysc+30}.nc')
        dsr = xr.open_dataset(f'{path_prace}/ECS/atm_{rcp}_{Q}_avg_2091_2101.nc')
        dsq = xr.open_dataset(f'{path_prace}/ECS/atm_{quad}_{Q}_avg_2041_2051.nc')
        if q==0:  # clouds
            for t, T in enumerate(['CLDLOW','CLDMED','CLDHGH','CLDTOT']):
                label = [['low clouds', 'medium clouds', 'high clouds', 'total clouds'][t], None][i]
                c = f'C{t}'
                control = dsc[T].mean('lon')
                ax[0,0].plot(control*100                   , dsc.lat, c=c, ls=ls, label=label)
                ax[0,1].plot((dsr[T].mean('lon')-control)*100, dsc.lat, c=c, ls=ls)
                ax[0,2].plot((dsq[T].mean('lon')-control)*100, dsc.lat, c=c, ls=ls)
        if q==1:  # radiation
            for t, T in enumerate(['FSNT','FSNTC','FLNT','FLNTC']):
                label = [['net SW', 'net SW clearsky', 'net LW', 'net LW clearsky'][t], None][i]
                c = f'C{t}'
                control = dsc[T].mean('lon')
                ax[1,0].plot(control                   , dsc.lat, c=c, ls=ls, label=label)
                ax[1,1].plot(dsr[T].mean('lon')-control, dsc.lat, c=c, ls=ls)
                ax[1,2].plot(dsq[T].mean('lon')-control, dsc.lat, c=c, ls=ls)
        if q==2:  # convection and precip
            T = 'PRECT'
            control = dsc[T].mean('lon')
            ax[2,0].plot(control                   , dsc.lat, ls=ls, label=['precipitation', None][i])
            ax[2,1].plot(dsr[T].mean('lon')-control, dsc.lat, ls=ls)
            ax[2,2].plot(dsq[T].mean('lon')-control, dsc.lat, ls=ls)
for i in range(3):
    ax[i,0].legend(fontsize=8)
    ax[i,0].set_ylim((-90,90))
    ax[i,0].set_yticks(np.arange(-90,91,30))


In [ ]:
ds = xr.open_mfdataset(fn_in, concat_dim='time').mean('time')
ds

In [ ]:
for q in ['CLDLOW','CLDMED','CLDHGH','CLDTOT']:
    ds[q].mean('lon').plot(label=q)
plt.legend()

In [ ]:
((ds.FSNT-ds.FSNTOA)/ds.FSNT).plot()